### C APIのPythonラッパーを定義する

In [ ]:
import ctypes
import numpy as np
from ctypes import c_char_p, c_bool, c_int, c_float, c_ubyte, POINTER, byref

# 加载动态库
cyolo = ctypes.cdll.LoadLibrary('./cmake-build-debug/libjetson.so')  # 或 'cyolo.dll' on Windows

# 函数原型定义
cyolo.c_yolo_init.argtypes = [c_char_p, c_bool]
cyolo.c_yolo_release.restype = c_bool
cyolo.c_yolo_add_image.argtypes = [c_int, POINTER(c_ubyte), c_int, c_int, c_int]
cyolo.c_yolo_add_image.restype = c_bool
cyolo.c_yolo_inference.restype = c_bool
cyolo.c_yolo_available_results.argtypes = [c_int, c_float, c_float]
cyolo.c_yolo_available_results.restype = c_int
cyolo.c_yolo_get_result.argtypes = [c_int, POINTER(c_int)]
cyolo.c_yolo_get_result.restype = POINTER(c_float)

# 封装成 Python 类
class YoloWrapper:
    def __init__(self, model_path: str, use_pose: bool = False):
        cyolo.c_yolo_init(model_path.encode('utf-8'), use_pose)

    def release(self):
        return cyolo.c_yolo_release()

    def add_image(self, index: int, image: np.ndarray):
        assert image.flags['C_CONTIGUOUS'], "Image must be C-contiguous"
        h, w, c = image.shape
        ptr = image.ctypes.data_as(POINTER(c_ubyte))
        return cyolo.c_yolo_add_image(index, ptr, c, w, h)

    def inference(self):
        return cyolo.c_yolo_inference()

    def available_results(self, index: int, cls_thresh: float, nms_thresh: float):
        return cyolo.c_yolo_available_results(index, cls_thresh, nms_thresh)

    def get_result(self, item_index: int):
        size = c_int()
        result_ptr = cyolo.c_yolo_get_result(item_index, byref(size))
        if not result_ptr:
            return None
        result_array = np.ctypeslib.as_array(result_ptr, shape=(size.value,))
        return result_array.copy()  # copy to detach from C memory

### YOLOのデータ構造を定義する

In [ ]:
import cv2

# Define the models
pose_model = YoloWrapper("/opt/models/yolov8n-pose.engine", use_pose=False)
object_model = YoloWrapper("/opt/models/yolov8n.engine", use_pose=False)

# Load the image
img = cv2.imread("image.jpg")  # HWC BGR
model.add_image(0, img)

model.inference()
count = model.available_results(0, 0.5, 0.45)

for i in range(count):
    res = model.get_result(i)
    print(res)

model.release()